In [ ]:
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 1.9 MB/s eta 0:00:00


In [ ]:

from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
#dataset = load_dataset("LucasThil/miniwob_actions_onhot")
#dataset = load_dataset("LucasThil/miniwob_snippets")
dataset = load_dataset("LucasThil/miniwob_snippets_refs_onehot")


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating validate split:   0%|          | 0/57976 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/58068 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/464060 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/LucasThil___parquet/LucasThil--miniwob_snippets_refs_onehot-f2c7b381a8320de6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# select how much of the datset we want to use
coef = 1
dataset['train'] = dataset["train"].select(range(int(len(dataset["train"])*coef)))
dataset['test'] = dataset["test"].select(range(int(len(dataset["test"])*coef)))
dataset['validate'] = dataset["validate"].select(range(int(len(dataset["validate"])*coef)))
dataset

DatasetDict({
    validate: Dataset({
        features: ['episodes', 'refs', 'click', 'dblclick', 'keydown', 'keypress', 'keyup', 'mousedown', 'mouseup', 'scroll', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '129', '130', '131', '132', '133', '134', '135', '136', '137'

In [ ]:
# We need to one-hot encode the refs, or have better predictions
refs_labels = sorted(list(set(dataset['train']['refs'] + dataset['test']['refs'] + dataset['validate']['refs'])))

stoi = {ch:i for i, ch in enumerate(refs_labels)}
itos = {i:ch for i, ch in enumerate(refs_labels)}

print(refs_labels)
print(len(refs_labels))

[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 32, 33, 35, 36, 38, 39, 41, 42, 44, 45, 46, 47, 50, 51, 54, 55, 59, 61, 62, 65, 67, 68, 69, 70, 71, 72, 75, 77, 78, 80, 81, 82, 83, 84, 87, 88, 91, 92, 93, 94, 95, 96, 97, 99, 100, 101, 102, 103, 104, 105, 106, 108, 109, 110, 112, 114, 117, 118, 121, 122, 123, 124, 125, 127, 129, 130, 134, 135, 138, 140, 142, 145, 146, 149, 152, 155]
104


In [ ]:
labels = [label for label in dataset['train'].features.keys() if label not in ['episodes', 'refs']]

# ==> add entries for ref targets
#labels = labels + refs_labels

id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

labels

['click',
 'dblclick',
 'keydown',
 'keypress',
 'keyup',
 'mousedown',
 'mouseup',
 'scroll',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '12

In [ ]:
print(id2label)
print(label2id)

{0: 'click', 1: 'dblclick', 2: 'keydown', 3: 'keypress', 4: 'keyup', 5: 'mousedown', 6: 'mouseup', 7: 'scroll', 8: '1', 9: '2', 10: '3', 11: '4', 12: '5', 13: '6', 14: '7', 15: '8', 16: '9', 17: '10', 18: '11', 19: '12', 20: '13', 21: '14', 22: '15', 23: '16', 24: '17', 25: '18', 26: '19', 27: '20', 28: '21', 29: '22', 30: '23', 31: '24', 32: '25', 33: '26', 34: '27', 35: '28', 36: '29', 37: '30', 38: '32', 39: '33', 40: '34', 41: '35', 42: '36', 43: '37', 44: '38', 45: '39', 46: '40', 47: '41', 48: '42', 49: '43', 50: '44', 51: '45', 52: '46', 53: '47', 54: '48', 55: '49', 56: '50', 57: '51', 58: '52', 59: '53', 60: '54', 61: '55', 62: '56', 63: '57', 64: '58', 65: '59', 66: '60', 67: '61', 68: '62', 69: '63', 70: '64', 71: '65', 72: '66', 73: '67', 74: '68', 75: '69', 76: '70', 77: '71', 78: '72', 79: '73', 80: '74', 81: '75', 82: '76', 83: '77', 84: '78', 85: '79', 86: '80', 87: '81', 88: '82', 89: '83', 90: '84', 91: '85', 92: '86', 93: '87', 94: '88', 95: '89', 96: '90', 97: '91',

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#tokenizer = AutoTokenizer.from_pretrained("t5-small")


def preprocess_data(examples):
  #print(f'examples: {examples}')
  # take a batch of texts
  text = examples["episodes"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)
  # add labels
  #print(f'example keys {example.keys()}')
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  #print(f'labels_batch {labels_batch}')
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/289 [00:00<?, ? examples/s]

Map:   0%|          | 0/290 [00:00<?, ? examples/s]

Map:   0%|          | 0/2320 [00:00<?, ? examples/s]

In [ ]:
example = encoded_dataset['train'][0]
print(example.keys())
print(example['labels'])
print(encoded_dataset['train'][0])

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])
{'input_ids': tensor([  101,  1063,  3145,  7698,  1010,  2403,  1065,  1063,  3145,  7698,
         1010,  2403,  1065,  1063,  3145,  7698,  1010,  2403,  1065,  1063,
         3145,

In [ ]:
tokenizer.decode(example['input_ids'])

'[CLS] { keydown, 14 } { keydown, 14 } { keydown, 14 } { keydown, 14 } { keydown, 14 } { keydown, 14 } { keydown, 14 } { keydown, 14 } { keydown, 14 } { keydown, 14 } { keydown, 14 } { keydown, 14 } { keydown, 14 } { keydown, 14 } { click, 14 } { click, 14 } { mouseup, 14 } { mouseup, 14 } { mousedown, 0 } { mousedown, 0 } { scroll, 0 } < body ref = " 0 " > < a ref = " 1 " > < div id = " starth1 " ref = " 3 " > < / div > < form id = " searchform " ref = " 5 " > < div group " ref = " 6 " > < / div > < div id = " geo - from - wrap " geo - wrap " ref = " 11 " > < label ref = " 12 " > < t text = " from " ref = " 13 " > < / t > < input _ text id = " geo - from " recordingtarget = " true " ref = " 14 " > < div ref = " 15 " > [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [ ]:
example['labels']

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['keydown', '14']

In [ ]:
encoded_dataset.set_format("torch")

In [ ]:
# Define Model

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
# Train the Model

In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [ ]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  1063,  3145,  7698,  1010,  2403,  1065,  1063,  3145,  7698,
         1010,  2403,  1065,  1063,  3145,  7698,  1010,  2403,  1065,  1063,
         3145,  7698,  1010,  2403,  1065,  1063,  3145,  7698,  1010,  2403,
         1065,  1063,  3145,  7698,  1010,  2403,  1065,  1063,  3145,  7698,
         1010,  2403,  1065,  1063,  3145,  7698,  1010,  2403,  1065,  1063,
         3145,  7698,  1010,  2403,  1065,  1063,  3145,  7698,  1010,  2403,
         1065,  1063,  3145,  7698,  1010,  2403,  1065,  1063,  3145,  7698,
         1010,  2403,  1065,  1063,  3145,  7698,  1010,  2403,  1065,  1063,
         3145,  7698,  1010,  2403,  1065,  1063, 11562,  1010,  2403,  1065,
         1063, 11562,  1010,  2403,  1065,  1063,  8000,  6279,  1010,  2403,
         1065,  1063,  8000,  6279,  1010,  2403,  1065,  1063,  8000,  7698,
         1010,  1014,  1065,  1063,  8000,  7698,  1010,  1014,  1065,  1063,
        17186,  1010,  1014,  1065,  1026,  2303, 25416,  1027, 

In [ ]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

TypeError: ignored

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validate"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/bert-finetuned-sem_eval-english is already a clone of https://huggingface.co/LucasThil/bert-finetuned-sem_eval-english. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 232030
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 29004
  Number of trainable parameters = 109609894
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:
Loading best model from bert-finetuned-sem_eval-english/checkpoint-5801 (score: 0.8556616115121857).
TrainOutput(global_step=5801, training_loss=0.040061643094446014, metrics={'train_runtime': 4795.5477, 'train_samples_per_second': 9.677, 'train_steps_per_second': 1.21, 'total_flos': 1.2227910635139072e+16, 'train_loss': 0.040061643094446014, 'epoch': 1.0})

# Check trained model

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import pipeline
from transformers import BertForSequenceClassification

#model = pipeline("fill-mask", model="LucasThil/bert-finetuned-sem_eval-english", tokenizer=tokenizer)
#model = AutoModelForSequenceClassification.from_pretrained("LucasThil/bert-finetuned-sem_eval-english")
model = BertForSequenceClassification.from_pretrained("LucasThil/bert-finetuned-sem_eval-english")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--LucasThil--bert-finetuned-sem_eval-english/snapshots/27944f31c29b3a50b28da3afcd1481a77dc47bba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "click",
    "1": "dblclick",
    "2": "keydown",
    "3": "keypress",
    "4": "keyup",
    "5": "mousedown",
    "6": "mouseup",
    "7": "scroll",
    "8": "1",
    "9": "2",
    "10": "3",
    "11": "4",
    "12": "5",
    "13": "6",
    "14": "7",
    "15": "8",
    "16": "9",
    "17": "10",
    "18": "11",
    "19": "12",
    "20": "13",
    "21": "14",
    "22": "15",
    "23": "16",
    "24": "17",
    "25": "18",
    "26": "19",
    "27": "20",
    "28": "2

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertModel.forward` and have been ignored: labels. If labels are not expected by `BertModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 289
  Batch size = 8
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_runtime': 679.5457,
 'eval_samples_per_second': 0.425,
 'eval_steps_per_second': 0.054}

In [ ]:
text = '{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{keydown,14}{click,14}{click,14}{mouseup,14}{mouseup,14}{mousedown,0}{mousedown,0}{scroll,0}<body ref="0"><a ref="1"><div id="starth1" ref="3"></DIV><form id="searchform" ref="5"><div group" ref="6"></DIV><div id="geo-from-wrap" geo-wrap" ref="11"><label ref="12"><t text="from" ref="13"></t><input_text id="geo-from" recordingtarget="true" ref="14"><div ref="15">'

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

In [ ]:
print(encoding)

{'input_ids': tensor([[  101,  1063,  3145,  7698,  1010,  2403,  1065,  1063,  3145,  7698,
          1010,  2403,  1065,  1063,  3145,  7698,  1010,  2403,  1065,  1063,
          3145,  7698,  1010,  2403,  1065,  1063,  3145,  7698,  1010,  2403,
          1065,  1063,  3145,  7698,  1010,  2403,  1065,  1063,  3145,  7698,
          1010,  2403,  1065,  1063,  3145,  7698,  1010,  2403,  1065,  1063,
          3145,  7698,  1010,  2403,  1065,  1063,  3145,  7698,  1010,  2403,
          1065,  1063,  3145,  7698,  1010,  2403,  1065,  1063,  3145,  7698,
          1010,  2403,  1065,  1063,  3145,  7698,  1010,  2403,  1065,  1063,
          3145,  7698,  1010,  2403,  1065,  1063, 11562,  1010,  2403,  1065,
          1063, 11562,  1010,  2403,  1065,  1063,  8000,  6279,  1010,  2403,
          1065,  1063,  8000,  6279,  1010,  2403,  1065,  1063,  8000,  7698,
          1010,  1014,  1065,  1063,  8000,  7698,  1010,  1014,  1065,  1063,
         17186,  1010,  1014,  1065,  

In [ ]:
logits = outputs.logits
logits.shape

torch.Size([1, 166])

In [ ]:
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-11.7089,  -8.7196,   3.0103,  -2.1103,  -4.5110,  -6.9212,  -6.9786,
          -8.4315, -11.3791, -11.1350, -12.6060,  -8.1280, -10.0994, -10.1783,
         -10.4578,  -8.6739,  -9.4633,  -9.3375,  -9.5148, -11.1789, -11.1517,
           7.4591,  -9.6926,  -9.9623, -10.1921, -10.6584, -12.9366,  -9.4359,
          -8.1682, -10.8048,  -9.4152, -10.9541, -11.3558, -11.3371,  -9.4254,
         -12.8403,  -9.9377,  -8.8363,  -9.8581, -11.4876, -11.2299, -10.0560,
          -9.5609, -12.1740,  -9.9502, -10.1650, -11.8595, -12.0295, -11.8895,
         -11.5470, -11.7961, -10.8603, -10.7563, -11.3330, -11.0095, -12.4675,
         -11.1132, -11.9535, -11.4510, -12.1450, -11.0551, -11.1244, -11.2237,
         -11.4000, -11.7275, -11.0454, -12.6766, -12.2165, -11.1350, -11.8353,
         -10.9130, -11.3394, -11.9149, -11.5028, -13.1119, -11.6596, -11.3417,
         -10.9926, -11.4760, -12.4369, -11.3817, -12.0144, -12.5088, -11.0247,
         

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['keydown', '14']


In [ ]:
print(probs)

[1. 0.]
